# Création d'un dictionnaire de tableaux pour chaque ligne de métro

In [1]:
%pip install openpyxl
import pandas as pd
import numpy as np
import re
import difflib
from IPython.utils import io
from IPython.display import display

Note: you may need to restart the kernel to use updated packages.


Importer le notebook Nettoyage données affluence par jour + nombre de validation par jour par station

In [2]:
try:
    with io.capture_output():
        %run -i "Nettoyage données affluence par jour + nombre de validation par jour par station.ipynb"
except Exception as e:
    display(e)

Importer les données disponibles sur le site https://data.ratp.fr .
Ce jeu de données présente le trafic des entrants directs sur le réseau ferré RATP en 2021

In [3]:
tableau_fréquentation = pd.read_excel('https://data.ratp.fr/api/explore/v2.1/catalog/datasets/trafic-annuel-entrant-par-station-du-reseau-ferre-2021/exports/xlsx?lang=fr&timezone=Europe%2FBerlin&use_labels=true')

print("Tableau importé : trafic des entrants directs sur le réseau ferré RATP en 2021")
display(tableau_fréquentation)


Tableau importé : trafic des entrants directs sur le réseau ferré RATP en 2021


,Rang,Réseau,Station,Trafic,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,Ville,Arrondissement pour Paris
0,3,Métro,GARE DE LYON,28640475,1,14,A,NaN,NaN,Paris,12.0
1,12,Métro,BELLEVILLE,7314438,2,11,NaN,NaN,NaN,Paris,10.0
2,32,Métro,OPERA,5193831,3,7,8,NaN,NaN,Paris,9.0
3,58,Métro,SAINT-MANDE-TOURELLE,3944640,1,NaN,NaN,NaN,NaN,Saint Mandé,NaN
4,59,Métro,HOCHE,3928404,5,NaN,NaN,NaN,NaN,Pantin,NaN
...,...,...,...,...,...,...,...,...,...,...,...
366,39,RER,FONTENAY-SOUS-BOIS,2121720,A,NaN,NaN,NaN,NaN,Fontenay-sous-Bois,NaN
367,40,RER,SUCY-BONNEUIL,1993503,A,NaN,NaN,NaN,NaN,Sucy-en-Brie,NaN
368,58,RER,LE GUICHET,530706,B,NaN,NaN,NaN,NaN,Orsay,NaN
369,65,RER,PARC DE SCEAUX,383626,B,NaN,NaN,NaN,NaN,Antony,NaN


Nettoyage du tableau

In [4]:
tableau_fréquentation = tableau_fréquentation[tableau_fréquentation['Réseau'] == 'Métro']
tableau_fréquentation = tableau_fréquentation.drop_duplicates(subset='Station')

colonnes_a_supprimer = ['Réseau', 'Ville', 'Arrondissement pour Paris']

tableau_fréquentation = tableau_fréquentation.drop(colonnes_a_supprimer, axis=1)

# Liste des colonnes à modifier
colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

# Convertir les colonnes spécifiées en type chaîne (si elles ne le sont pas déjà)
tableau_fréquentation[colonnes_correspondance] = tableau_fréquentation[colonnes_correspondance].astype(str)

# Supprimer les lettres 'A' et 'B' des colonnes spécifiées dans tableau_fréquentation
for colonne in colonnes_correspondance:
    tableau_fréquentation[colonne] = tableau_fréquentation[colonne].str.replace('A', '').str.replace('B', '')

display(tableau_fréquentation)

,Rang,Station,Trafic,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5
0,3,GARE DE LYON,28640475,1,14,,nan,nan
1,12,BELLEVILLE,7314438,2,11,nan,nan,nan
2,32,OPERA,5193831,3,7,8,nan,nan
3,58,SAINT-MANDE-TOURELLE,3944640,1,nan,nan,nan,nan
4,59,HOCHE,3928404,5,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...
350,272,SULLY-MORLAND,1124169,7,nan,nan,nan,nan
351,275,EUROPE,1106459,3,nan,nan,nan,nan
352,283,CAMPO-FORMIO,1016150,5,nan,nan,nan,nan
353,298,SAINT-FARGEAU,472258,3bis,nan,nan,nan,nan


In [5]:
# Ajouter la colonne 'nb_correspondance'
tableau_fréquentation['nb_correspondance'] = tableau_fréquentation[colonnes_correspondance].apply(lambda row: sum(pd.notna(value) and value != 'nan' for value in row), axis=1)

# Diviser les valeurs de la colonne "Trafic" par les valeurs de la colonne "nb_correspondance"
tableau_fréquentation['Trafic'] = tableau_fréquentation['Trafic'] / tableau_fréquentation['nb_correspondance']

display(tableau_fréquentation)

,Rang,Station,Trafic,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,nb_correspondance
0,3,GARE DE LYON,9546825.0,1,14,,nan,nan,3
1,12,BELLEVILLE,3657219.0,2,11,nan,nan,nan,2
2,32,OPERA,1731277.0,3,7,8,nan,nan,3
3,58,SAINT-MANDE-TOURELLE,3944640.0,1,nan,nan,nan,nan,1
4,59,HOCHE,3928404.0,5,nan,nan,nan,nan,1
...,...,...,...,...,...,...,...,...,...
350,272,SULLY-MORLAND,1124169.0,7,nan,nan,nan,nan,1
351,275,EUROPE,1106459.0,3,nan,nan,nan,nan,1
352,283,CAMPO-FORMIO,1016150.0,5,nan,nan,nan,nan,1
353,298,SAINT-FARGEAU,472258.0,3bis,nan,nan,nan,nan,1


Création d'un dictionnaire contenant les fréquentations de chaque station de chaque ligne 

In [6]:
numéro = [1, 2, 3, '3bis', 4, 5, 6, 7, '7bis', 8, 9, 10, 11, 12, 13, 14]

# Création des tableaux pour chaque élément de numéro
tableaux = {}

colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

for k in numéro:
    # Convertir k en string car il peut contenir des valeurs non numériques
    k_str = str(k)
    
    # Filtrer les lignes où la valeur k est présente dans les colonnes spécifiées
    filtered_rows = tableau_fréquentation[tableau_fréquentation[colonnes_correspondance].apply(lambda x: k_str in str(x.values), axis=1)]
    
    # Supprimer les doublons dans la colonne 'Station'
    filtered_rows = filtered_rows.drop_duplicates(subset='Station')
    
    # Stocker les lignes filtrées dans un tableau nommé "ligne numéro[k]"
    tableaux[f'ligne numéro[{k_str}]'] = filtered_rows

# Parcourir chaque tableau dans le dictionnaire et supprimer les colonnes spécifiées
for tableau_nom, tableau in tableaux.items():
    tableaux[tableau_nom] = tableau.drop(columns=colonnes_correspondance)
    tableau = tableau.drop_duplicates(subset='Station')
    tableaux[tableau_nom] = tableau.drop_duplicates(subset='Station')  # Ajoutez cette ligne pour supprimer les doublons de la colonne 'Station'


In [7]:

# Parcourir chaque tableau dans le dictionnaire
for tableau_nom, tableau in tableaux.items():
    # Calculer la fréquentation totale pour chaque tableau
    fréquentation_totale_ligne = tableau['Trafic'].sum()

    # Ajouter la variable fréquentation_totale à chaque tableau
    tableaux[tableau_nom]['fréquentation_totale_ligne'] = fréquentation_totale_ligne
    tableau['Part du trafic de la station par rapport à toute la ligne'] = tableau['Trafic'] / tableau['fréquentation_totale_ligne']

# Supposons que la clé pour la correspondance soit 'LIBELLE_ARRET' dans le tableau moyenne_par_arret
cle_correspondance = 'LIBELLE_ARRET'

# Nettoyage des espaces et uniformisation de la casse
moyenne_par_arret['LIBELLE_ARRET'] = moyenne_par_arret['LIBELLE_ARRET'].str.strip().str.lower()
for tableau_nom, tableau in tableaux.items():
    tableau['Station'] = tableau['Station'].str.strip().str.lower()

    # Fusion en utilisant les noms de stations nettoyés
    merged_table = pd.merge(tableau, moyenne_par_arret, left_on='Station', right_on='LIBELLE_ARRET', how='left')
    
    # Renommer la colonne NB_VALD de moyenne_par_arret
    merged_table = merged_table.rename(columns={'NB_VALD': 'fréquentation moyenne hors JO'})
    
    tableaux[tableau_nom] = merged_table

In [8]:
#compléter avec les valeurs manquantes

# Méthode pour trouver la correspondance la plus proche
def find_closest_match(target, candidates):
    closest_match = difflib.get_close_matches(target, candidates, n=1, cutoff=0.4)  # Ajustez le seuil selon vos besoins
    return closest_match[0] if closest_match else None

for k in numéro:
    k_str = str(k)
    # Accéder aux tableaux spécifiques
    current_table = tableaux[f'ligne numéro[{k_str}]']
    
    # Filtrer les lignes avec des valeurs manquantes dans la colonne 'LIBELLE_ARRET'
    missing_data = current_table[current_table['LIBELLE_ARRET'].isnull()]
    
    # Remplir les valeurs manquantes
    for index, row in missing_data.iterrows():
        closest_match = find_closest_match(row['Station'], moyenne_par_arret['LIBELLE_ARRET'])
        
        if closest_match:
            tableaux[f'ligne numéro[{k_str}]'].at[index, 'LIBELLE_ARRET'] = closest_match



In [9]:
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Fusionner avec le tableau moyenne_par_arret pour obtenir les valeurs NB_VALD associées
    merged_with_nb_vald = pd.merge(current_table, moyenne_par_arret[['LIBELLE_ARRET', 'NB_VALD']],
                                   left_on='LIBELLE_ARRET', right_on='LIBELLE_ARRET', how='left')
    
    # Renommer la colonne NB_VALD pour plus de clarté
    merged_with_nb_vald = merged_with_nb_vald.rename(columns={'NB_VALD': 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'})

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = merged_with_nb_vald

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Fusionner les données avec le tableau contenant les nouvelles colonnes
    merged_table = pd.merge(current_table, pivot_table, on='LIBELLE_ARRET', how='left')

    # Mettre à jour le tableau dans le dictionnaire avec les nouvelles colonnes fusionnées
    tableaux[f'ligne numéro[{k_str}]'] = merged_table

In [10]:
for k in numéro:
    k_str = str(k)
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Supprimer les doublons de stations
    current_table = current_table.drop_duplicates(subset='Station')

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = current_table


In [11]:
for k in numéro:
    k_str = str(k)
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Supprimer les colonnes 'fréquentation moyenne hors JO' et 'LIBELLE_ARRET' si elles existent
    columns_to_drop = ['fréquentation moyenne hors JO', 'LIBELLE_ARRET']
    current_table = current_table.drop(columns=columns_to_drop, errors='ignore')

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = current_table

In [12]:
for k in numéro:
    k_str = str(k)
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si les colonnes existent avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station' in current_table.columns and 'nb_correspondance' in current_table.columns:
        # Calcul de la nouvelle colonne
        current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'] / current_table['nb_correspondance']

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table

for k in numéro:
    k_str = str(k)
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si les colonnes existent avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station' in current_table.columns and 'nb_correspondance' in current_table.columns:
        # Calcul de la nouvelle colonne
        current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'] / current_table['nb_correspondance'] / 2

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si la colonne existe avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne' in current_table.columns:
        # Calculer la somme de la colonne spécifique
        current_table['nombre_validations_totales_pour_la_ligne'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne'].sum()

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table


In [13]:
# Définir le nouvel ordre des colonnes
nouvel_ordre_colonnes = [
    'Station',
    'Trafic',
    'Part du trafic de la station par rapport à toute la ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction', 'nb_correspondance',
    'Correspondance_1',
    'Correspondance_2',
    'Correspondance_3',
    'Correspondance_4',
    'Correspondance_5',
    'nombre_validations_totales_pour_la_ligne',
    'fréquentation_totale_ligne',
    '00:00-01:00',
    '01:00-02:00',
    '02:00-03:00',
    '03:00-04:00',
    '04:00-05:00',
    '05:00-06:00',
    '06:00-07:00',
    '07:00-08:00',
    '08:00-09:00',
    '09:00-10:00',
    '10:00-11:00',
    '11:00-12:00',
    '12:00-13:00',
    '13:00-14:00',
    '14:00-15:00',
    '15:00-16:00',
    '16:00-17:00',
    '17:00-18:00',
    '18:00-19:00',
    '19:00-20:00',
    '20:00-21:00',
    '21:00-22:00',
    '22:00-23:00',
    '23:00-00:00',
    'Rang'
]

# Appliquer le nouvel ordre des colonnes à chaque tableau
for tableau_nom, tableau in tableaux.items():
    tableaux[tableau_nom] = tableau[nouvel_ordre_colonnes]

# Vérifier que rien n'a été oublié
print("Colonnes disponibles :")
print(tableau.columns)
for tableau_nom, tableau in tableaux.items():
    print(f"Tableau: {tableau_nom}")
    
    # Réorganiser les colonnes selon le nouvel ordre
    tableau = tableau[nouvel_ordre_colonnes]
    

Colonnes disponibles :
Index(['Rang', 'Station', 'Trafic', 'Correspondance_1', 'Correspondance_2',
       'Correspondance_3', 'Correspondance_4', 'Correspondance_5',
       'nb_correspondance', 'fréquentation_totale_ligne',
       'Part du trafic de la station par rapport à toute la ligne',
       'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station',
       '00:00-01:00', '01:00-02:00', '02:00-03:00', '03:00-04:00',
       '04:00-05:00', '05:00-06:00', '06:00-07:00', '07:00-08:00',
       '08:00-09:00', '09:00-10:00', '10:00-11:00', '11:00-12:00',
       '12:00-13:00', '13:00-14:00', '14:00-15:00', '15:00-16:00',
       '16:00-17:00', '17:00-18:00', '18:00-19:00', '19:00-20:00',
       '20:00-21:00', '21:00-22:00', '22:00-23:00', '23:00-00:00',
       'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne',
       'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction',
       'nombre_validations_totales_pour_la_ligne'],
      dtype='object')
Tableau: ligne numéro[1]
Tableau

In [14]:
# Définir le nouvel ordre des colonnes
nouvel_ordre_colonnes = [
    'Station',
    'Trafic', 
    'Part du trafic de la station par rapport à toute la ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction', 'nb_correspondance',
    'Correspondance_1',
    'Correspondance_2',
    'Correspondance_3',
    'Correspondance_4',
    'Correspondance_5',
    'nombre_validations_totales_pour_la_ligne','fréquentation_totale_ligne',
    '00:00-01:00',
    '01:00-02:00',
    '02:00-03:00',
    '03:00-04:00',
    '04:00-05:00',
    '05:00-06:00',
    '06:00-07:00',
    '07:00-08:00',
    '08:00-09:00',
    '09:00-10:00',
    '10:00-11:00',
    '11:00-12:00',
    '12:00-13:00',
    '13:00-14:00',
    '14:00-15:00',
    '15:00-16:00',
    '16:00-17:00',
    '17:00-18:00',
    '18:00-19:00',
    '19:00-20:00',
    '20:00-21:00',
    '21:00-22:00',
    '22:00-23:00',
    '23:00-00:00',
    'Rang'
]

# Appliquer le nouvel ordre des colonnes à chaque tableau
for tableau_nom, tableau in tableaux.items():
    tableaux[tableau_nom] = tableau[nouvel_ordre_colonnes]

# Vérifier que rien n'a été oublié
print("Colonnes disponibles :")


Colonnes disponibles :


In [15]:
# Nouveaux noms des colonnes
nouveaux_noms_colonnes = {
    'Station': 'Station',
    'Trafic': 'Trafic annuel dans la station',
    'Part du trafic de la station par rapport à toute la ligne': 'Part de la station dans le trafic annuel de la ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station': 'Moyenne de la fréquentation journalière de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne': 'Moyenne de la fréquentation journalière de chaque correspondance de la station en 2022, sur la période des JO',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction': 'Moyenne de la fréquentation journalière de chaque correspondance, pour chaque direction en 2022, sur la période des JO',
    'Correspondance_1': 'Correspondance 1',
    'Correspondance_2': 'Correspondance 2',
    'Correspondance_3': 'Correspondance 3',
    'Correspondance_4': 'Correspondance 4',
    'Correspondance_5': 'Correspondance 5',
    'nb_correspondance': 'Nombre de correspondances de la station',
    'nombre_validations_totales_pour_la_ligne': 'Nombre de validations annuelles sur la ligne',
    'fréquentation_totale_ligne': 'Fréquentation totale de la ligne',
    '00:00-01:00': '00:00-01:00',
    '01:00-02:00': '01:00-02:00',
    '02:00-03:00': '02:00-03:00',
    '03:00-04:00': '03:00-04:00',
    '04:00-05:00': '04:00-05:00',
    '05:00-06:00': '05:00-06:00',
    '06:00-07:00': '06:00-07:00',
    '07:00-08:00': '07:00-08:00',
    '08:00-09:00': '08:00-09:00',
    '09:00-10:00': '09:00-10:00',
    '10:00-11:00': '10:00-11:00',
    '11:00-12:00': '11:00-12:00',
    '12:00-13:00': '12:00-13:00',
    '13:00-14:00': '13:00-14:00',
    '14:00-15:00': '14:00-15:00',
    '15:00-16:00': '15:00-16:00',
    '16:00-17:00': '16:00-17:00',
    '17:00-18:00': '17:00-18:00',
    '18:00-19:00': '18:00-19:00',
    '19:00-20:00': '19:00-20:00',
    '20:00-21:00': '20:00-21:00',
    '21:00-22:00': '21:00-22:00',
    '22:00-23:00': '22:00-23:00',
    '23:00-00:00': '23:00-00:00',
    'Rang': 'Rang'
}

# Afficher chaque tableau final avec le nouvel ordre des colonnes et les nouveaux noms
for tableau_nom, tableau in tableaux.items():
    print(f"Tableau: {tableau_nom}")
    
    # Réorganiser les colonnes selon le nouvel ordre
    tableau = tableau[nouvel_ordre_colonnes]
    
    # Renommer les colonnes
    tableau = tableau.rename(columns=nouveaux_noms_colonnes)
    
    display(tableau)
    print("\n" + "=" * 50 + "\n")


Tableau: ligne numéro[1]


,Station,Trafic annuel dans la station,Part de la station dans le trafic annuel de la ligne,"Moyenne de la fréquentation journalière de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)","Moyenne de la fréquentation journalière de chaque correspondance de la station en 2022, sur la période des JO","Moyenne de la fréquentation journalière de chaque correspondance, pour chaque direction en 2022, sur la période des JO",Nombre de correspondances de la station,Correspondance 1,Correspondance 2,Correspondance 3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,gare de lyon,9546825.0,0.031973,5298.853444,1766.284481,883.142241,3,1,14,,...,34.76,45.22,57.99,60.21,38.73,27.60,17.40,10.24,4.80,3
2,belleville,3657219.0,0.012248,2329.793584,1164.896792,582.448396,2,2,11,nan,...,38.84,39.88,41.62,37.49,32.01,21.30,14.72,13.35,11.97,12
3,saint-mande-tourelle,3944640.0,0.013211,1321.871981,1321.871981,660.935990,1,1,nan,nan,...,35.69,37.54,44.98,47.84,34.26,19.04,10.38,8.05,5.99,58
4,bercy,1942106.0,0.006504,1407.759105,703.879553,351.939776,2,6,14,nan,...,2.37,109.81,104.32,67.23,40.18,13.57,0.11,14.23,0.09,61
6,saint-denis-universite,3569990.0,0.011956,1306.047187,1306.047187,653.023593,1,13,nan,nan,...,30.33,28.90,31.51,27.77,18.64,13.60,8.89,6.17,4.36,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,michel-ange-auteuil,756025.0,0.002532,512.335966,256.167983,128.083991,2,9,10,nan,...,37.63,40.86,43.35,40.46,35.29,21.50,11.78,11.57,9.71,234
124,notre-dame-des-champs,1487256.0,0.004981,510.183971,510.183971,255.091985,1,12,nan,nan,...,36.70,46.97,51.64,51.20,38.02,25.48,14.76,14.98,12.61,238
125,malakoff-rue etienne dolet,1450451.0,0.004858,487.442831,487.442831,243.721416,1,13,nan,nan,...,33.51,34.59,43.10,41.24,28.00,15.65,8.94,6.14,4.88,239
126,michel-ange-molitor,710276.0,0.002379,525.684498,262.842249,131.421125,2,9,10,nan,...,37.02,39.89,42.74,38.67,35.84,19.83,11.89,10.72,13.92,243




Tableau: ligne numéro[2]


,Station,Trafic annuel dans la station,Part de la station dans le trafic annuel de la ligne,"Moyenne de la fréquentation journalière de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)","Moyenne de la fréquentation journalière de chaque correspondance de la station en 2022, sur la période des JO","Moyenne de la fréquentation journalière de chaque correspondance, pour chaque direction en 2022, sur la période des JO",Nombre de correspondances de la station,Correspondance 1,Correspondance 2,Correspondance 3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,belleville,3.657219e+06,0.031606,2329.793584,1164.896792,582.448396,2,2,11,nan,...,38.84,39.88,41.62,37.49,32.01,21.30,14.72,13.35,11.97,12
1,colonel fabien,3.043606e+06,0.026303,961.150485,961.150485,480.575243,1,2,nan,nan,...,35.77,39.02,45.10,41.33,34.81,22.89,15.25,12.20,10.86,104
2,menilmontant,2.847264e+06,0.024606,885.649272,885.649272,442.824636,1,2,nan,nan,...,32.69,34.55,37.85,37.05,33.65,24.10,18.15,17.53,17.79,118
3,blanche,2.167570e+06,0.018732,628.054612,628.054612,314.027306,1,2,nan,nan,...,34.19,37.46,41.29,41.39,36.88,27.88,21.15,27.09,24.68,163
4,rue du bac,1.423364e+06,0.012301,511.815741,511.815741,255.907871,1,12,nan,nan,...,40.07,50.13,57.77,57.82,48.87,23.13,15.46,12.09,11.93,242
5,philippe auguste,1.262653e+06,0.010912,385.908318,385.908318,192.954159,1,2,nan,nan,...,36.68,41.25,49.12,42.83,32.67,21.55,13.32,12.08,9.82,256
6,monceau,1.148677e+06,0.009927,365.040097,365.040097,182.520049,1,2,nan,nan,...,39.02,48.87,62.67,65.78,47.89,22.54,11.59,7.52,5.00,268
7,rennes,8.613340e+05,0.007444,298.964763,298.964763,149.482382,1,12,nan,nan,...,44.08,52.01,58.20,56.48,44.05,23.63,10.44,10.24,8.72,288
8,montparnasse-bienvenue,5.101806e+06,0.044090,8840.644431,2210.161108,1105.080554,4,4,6,12,...,26.18,36.84,55.50,62.72,49.65,31.05,21.54,16.83,13.49,4
10,madeleine,1.776976e+06,0.015357,1850.791063,616.930354,308.465177,3,8,12,14,...,43.85,53.91,65.73,71.49,60.81,23.76,11.64,11.65,10.10,27




Tableau: ligne numéro[3]


...

,Station,Trafic annuel dans la station,Part de la station dans le trafic annuel de la ligne,"Moyenne de la fréquentation journalière de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)","Moyenne de la fréquentation journalière de chaque correspondance de la station en 2022, sur la période des JO","Moyenne de la fréquentation journalière de chaque correspondance, pour chaque direction en 2022, sur la période des JO",Nombre de correspondances de la station,Correspondance 1,Correspondance 2,Correspondance 3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,rue du bac,1.423364e+06,0.023459,511.815741,511.815741,255.907871,1,12,nan,nan,...,40.07,50.13,57.77,57.82,48.87,23.13,15.46,12.09,11.93,242
1,rennes,8.613340e+05,0.014196,298.964763,298.964763,149.482382,1,12,nan,nan,...,44.08,52.01,58.20,56.48,44.05,23.63,10.44,10.24,8.72,288
2,montparnasse-bienvenue,5.101806e+06,0.084085,8840.644431,2210.161108,1105.080554,4,4,6,12,...,26.18,36.84,55.50,62.72,49.65,31.05,21.54,16.83,13.49,4
4,madeleine,1.776976e+06,0.029287,1850.791063,616.930354,308.465177,3,8,12,14,...,43.85,53.91,65.73,71.49,60.81,23.76,11.64,11.65,10.10,27
5,sevres-babylone,1.696252e+06,0.027957,1213.443299,606.721649,303.360825,2,10,12,nan,...,39.67,51.13,59.22,66.54,57.85,35.27,12.25,9.71,8.96,88
6,mairie d'issy,2.874138e+06,0.047370,979.186173,979.186173,489.593087,1,12,nan,nan,...,34.97,36.23,38.97,38.91,30.74,16.37,8.48,6.66,5.20,115
7,notre-dame-de-lorette,1.893828e+06,0.031213,639.111922,639.111922,319.555961,1,12,nan,nan,...,35.01,35.65,44.18,49.69,39.57,25.65,17.51,18.02,18.33,188
8,porte de versailles,3.268157e+06,0.053864,2411.033293,2411.033293,1205.516646,1,12,nan,nan,...,39.11,50.70,60.08,62.00,53.19,32.02,19.18,19.01,18.49,93
9,vaugirard,2.482886e+06,0.040921,785.489982,785.489982,392.744991,1,12,nan,nan,...,37.27,41.28,44.22,41.04,34.89,19.16,10.52,9.18,7.66,136
10,marx dormoy,2.425928e+06,0.039983,792.976815,792.976815,396.488408,1,12,nan,nan,...,36.82,37.68,43.01,38.88,31.06,21.78,14.49,10.47,8.05,139




Tableau: ligne numéro[13]


,Station,Trafic annuel dans la station,Part de la station dans le trafic annuel de la ligne,"Moyenne de la fréquentation journalière de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)","Moyenne de la fréquentation journalière de chaque correspondance de la station en 2022, sur la période des JO","Moyenne de la fréquentation journalière de chaque correspondance, pour chaque direction en 2022, sur la période des JO",Nombre de correspondances de la station,Correspondance 1,Correspondance 2,Correspondance 3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,saint-denis-universite,3569990.0,0.042992,1306.047187,1306.047187,653.023593,1,13,nan,nan,...,30.33,28.90,31.51,27.77,18.64,13.60,8.89,6.17,4.36,74
1,les courtilles,2802689.0,0.033751,1071.203760,1071.203760,535.601880,1,13,nan,nan,...,30.03,31.76,35.24,31.76,23.70,16.18,10.77,7.34,4.88,122
2,porte de saint-ouen,2710638.0,0.032643,756.074953,756.074953,378.037477,1,13,nan,nan,...,34.06,38.01,42.89,40.38,34.53,23.47,16.62,12.24,9.27,125
3,montparnasse-bienvenue,5101806.0,0.061438,8840.644431,2210.161108,1105.080554,4,4,6,12,...,26.18,36.84,55.50,62.72,49.65,31.05,21.54,16.83,13.49,4
5,porte de clichy,2639248.5,0.031783,1929.658213,964.829106,482.414553,2,13,14,nan,...,67.67,75.58,92.43,88.85,71.86,46.94,32.62,24.38,18.66,29
6,place clichy,2580966.0,0.031081,1680.341596,840.170798,420.085399,2,2,13,nan,...,34.37,37.23,41.68,40.48,38.04,26.06,20.67,21.62,21.26,33
7,porte de vanves,3395358.0,0.040889,1143.166063,1143.166063,571.583031,1,13,nan,nan,...,32.51,35.64,41.18,39.19,30.95,21.34,13.95,10.18,8.11,87
8,pernety,2173567.0,0.026175,692.335762,692.335762,346.167881,1,13,nan,nan,...,33.91,33.31,34.66,34.29,31.53,21.05,13.67,12.71,10.96,162
9,carrefour pleyel,1535528.0,0.018492,557.921485,557.921485,278.960743,1,13,nan,nan,...,33.97,48.79,57.68,45.30,28.87,16.22,10.19,8.39,6.78,230
10,gabriel peri,3790573.0,0.045648,1262.509411,1262.509411,631.254706,1,13,nan,nan,...,31.95,34.54,38.88,38.49,29.48,18.40,11.34,7.23,5.40,64




Tableau: ligne numéro[14]


,Station,Trafic annuel dans la station,Part de la station dans le trafic annuel de la ligne,"Moyenne de la fréquentation journalière de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)","Moyenne de la fréquentation journalière de chaque correspondance de la station en 2022, sur la période des JO","Moyenne de la fréquentation journalière de chaque correspondance, pour chaque direction en 2022, sur la période des JO",Nombre de correspondances de la station,Correspondance 1,Correspondance 2,Correspondance 3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,gare de lyon,9546825.0,0.172042,5298.853444,1766.284481,883.142241,3,1,14,,...,34.76,45.22,57.99,60.21,38.73,27.60,17.40,10.24,4.80,3
2,bercy,1942106.0,0.034998,1407.759105,703.879553,351.939776,2,6,14,nan,...,2.37,109.81,104.32,67.23,40.18,13.57,0.11,14.23,0.09,61
4,madeleine,1776976.0,0.032023,1850.791063,616.930354,308.465177,3,8,12,14,...,43.85,53.91,65.73,71.49,60.81,23.76,11.64,11.65,10.10,27
5,porte de clichy,2639248.5,0.047561,1929.658213,964.829106,482.414553,2,13,14,nan,...,67.67,75.58,92.43,88.85,71.86,46.94,32.62,24.38,18.66,29
6,cour saint-emilion,2985122.0,0.053794,1084.386594,1084.386594,542.193297,1,14,nan,nan,...,36.59,49.66,61.54,60.91,45.62,29.16,25.69,15.63,12.76,110
7,bibliotheque,11104474.0,0.200112,2272.627107,2272.627107,1136.313554,1,14,nan,nan,...,35.33,43.36,54.02,56.83,44.83,26.99,18.84,11.58,8.76,6
8,chatelet,1670158.8,0.030098,5877.878321,1175.575664,587.787832,5,1,4,7,...,72.79,92.19,114.31,122.78,105.60,70.85,51.84,47.85,44.60,10
9,olympiades,5214595.0,0.093971,1895.432969,1895.432969,947.716484,1,14,nan,nan,...,38.44,42.90,47.26,41.98,31.78,20.50,13.19,6.90,4.47,31
10,saint-lazare,6625676.8,0.119400,18616.056763,3723.211353,1861.605676,5,3,9,12,...,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86,2
12,mairie de saint-ouen,2415405.0,0.043528,846.124381,423.062191,211.531095,2,13,14,nan,...,67.36,77.62,93.80,89.63,63.45,40.80,24.30,16.35,13.66,39


Enregistrer les tableaux

In [16]:
for k in numéro:
    k_str = str(k)
    tableaux[f'ligne numéro[{k_str}]'].to_csv(f'ligne_numero_{k_str}.csv', index=False) 